In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as T
import keras
from keras import backend as K
from keras import initializers
from keras.initializers import RandomNormal
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, merge, Reshape,  Flatten, Dropout
from keras.optimizers import Adagrad, Adam, SGD, RMSprop, Adamax
from keras.regularizers import l2
from keras.layers import Multiply, Concatenate
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from time import time
import multiprocessing as mp
import sys
import math
import argparse
import matplotlib.pyplot as plt

!python -m pip install -U -q PyDrive ## you will have install for every colab session

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
MOVIELENS_DIR = 'ml-100k'
RATING_DATA_FILE = 'u.data'
RATING_DATA_GEN_FILE = 'u_gen.data'
RATINGS_CSV_FILE = 'u_emb.data'
RATINGS_CSV_FILE_NORM = 'u_norm.data'
RATINGS_GEN_CSV_FILE = 'u_genr.data'
MODEL_WEIGHTS_FILE = 'u_emb_weights.h5'
MODEL_WEIGHTS_FILE_CORE = 'u_emb_weights'

In [ ]:
def get_ncf_model(num_users, num_items, latent_dim,hidden_dim,do):  # todo: this is from Asi!!!
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', input_length=1)   
    
    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings
    conc = Concatenate()([user_latent, item_latent])
    drop = Dropout(0.5)(conc)
    hid1 = Dense(hidden_dim, activation='relu')(conc)
    drop2  = Dropout(do)(hid1)
    prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'prediction')(drop2)
    
    
    model = Model(input=[user_input, item_input], output=prediction)
    print("ncf model")
    model.summary()

    return model

In [ ]:
# # create the model
# # Input variables
# user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
# item_input = Input(shape=(1,), dtype='int32', name = 'item_input')
# 
# num_users=len(users)
# num_items=len(genre)
# MF_Embedding_User = Embedding(input_dim = num_users, output_dim = 1, name = 'user_embedding', input_length=1)
# MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = 1, name = 'item_embedding', input_length=1)
# 
# # Crucial to flatten an embedding vector!
# user_latent = Flatten()(MF_Embedding_User(user_input))
# item_latent = Flatten()(MF_Embedding_Item(item_input))
# 
# # Element-wise product of user and item embeddings
# conc = Concatenate()([user_latent, item_latent])
# drop = Dropout(0.5)(conc)
# prediction = Dense(1, activation='relu', kernel_initializer='lecun_uniform', name = 'prediction')(drop)
# 
# NCF_model = Model(input=[user_input, item_input], output=prediction)
# NCF_model.summary()
# 
# NCF_model.compile(loss='mse',optimizer=Adamax(lr=0.001),metrics=['mae'])